# 2.2. Advanced NumPy


In [1]:
import numpy as np

NumPy is at the base of Python’s scientific stack of tools. Its purpose to implement efficient operations on many items in a block of memory. Understanding how it works in detail helps in making efficient use of its flexibility, taking useful shortcuts.

This section covers:

- Anatomy of NumPy arrays, and its consequences. Tips and tricks.
- Universal functions: what, why, and what to do if you want a new one.
- Integration with other tools: NumPy offers several ways to wrap any data in an ndarray, without unnecessary copies.

## 2.2.1. Life of ndarray

```
ndarray =

    block of memory + indexing scheme + data type descriptor

        raw data
        how to locate an element
        how to interpret an element


```

```c
typedef struct PyArrayObject {
        PyObject_HEAD

        /* Block of memory */
        char *data;

        /* Data type descriptor */
        PyArray_Descr *descr;

        /* Indexing scheme */
        int nd;
        npy_intp *dimensions;
        npy_intp *strides;

        /* Other stuff */
        PyObject *base;
        int flags;
        PyObject *weakreflist;
} PyArrayObject;

```

### 2.2.1.2. Block of memory

In [2]:
x = np.array([1, 2, 3], dtype=np.int32)
x.data

In [3]:
bytes(x.data)

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'

In [4]:
# memory address of the data
x.__array_interface__['data'][0]

1834343561904

The whole `__array_interface__`:

In [5]:
x.__array_interface__

{'data': (1834343561904, False),
 'strides': None,
 'descr': [('', '<i4')],
 'typestr': '<i4',
 'shape': (3,),
 'version': 3}

Reminder: two ndarrays may share the same memory:

In [6]:
x = np.array([1, 2, 3, 4])
y = x[:-1]
x[0] = 9
y

array([9, 2, 3])

Memory does not need to be owned by an ndarray:

In [7]:
x = b'1234'

x is a string (in Python 3 a bytes), we can represent its data as an array of ints:

In [8]:
y = np.frombuffer(x, dtype=np.int8)
y.data

In [9]:
y.base is x

True

In [10]:
y.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : False
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

### 2.2.1.3. Data types

`dtype` describes a single item in the array:

|    |     |
|----|-----|
|type|scalar type of the data, one of:
||int8, int16, float64, et al. (fixed size)
||str, unicode, void (flexible size)
|itemsize| size of the data block|
|byteorder|byte order: big-endian > / little-endian < / not applicable|
|fields|sub-types, if it's a structured data type|
|shape|shape of the array, if its a sub-array|

In [11]:
np.dtype(int).type

numpy.int32

In [12]:
np.dtype(int).itemsize

4

In [13]:
np.dtype(int).byteorder

'='

#### Casting and re-interpretation/views


casting

 - on assignment
 - on array construction
 - on arithmetic
 - etc.
 - and manually: .astype(dtype)

data re-interpretation

 - manually: .view(dtype)



- Casting in arithmetic, in nutshell:
    - only type (not value!) of operands matters
    - largest “safe” type able to represent both is picked
    - scalars can “lose” to arrays in some situations

Casting in general copies data:

In [14]:
x = np.array([1, 2, 3, 4], dtype=np.float)
x

array([1., 2., 3., 4.])

In [15]:
y = x.astype(np.int8)
y

array([1, 2, 3, 4], dtype=int8)

In [16]:
y + 1

array([2, 3, 4, 5], dtype=int8)

In [17]:
y + 256

array([257, 258, 259, 260], dtype=int16)

In [18]:
y + 256.0

array([257., 258., 259., 260.])

In [19]:
y + np.array([256], dtype=np.int32)

array([257, 258, 259, 260])

Casting on setitem: dtype of the array is not changed on item assignment:

In [20]:
y[:] = y + 1.5
y

array([2, 3, 4, 5], dtype=int8)

### 2.2.1.4. Indexing scheme: strides

At which byte in ``x.data`` does the item ``x[1, 2]`` begin?

In [21]:
x = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]], dtype=np.int8)
x.tobytes('A')

b'\x01\x02\x03\x04\x05\x06\x07\x08\t'

he answer (in NumPy)

- strides: the number of bytes to jump to find the next element
- 1 stride per dimension



In [22]:
x.strides

(3, 1)

In [23]:
byte_offset = 3*1 + 1*2
x.flat[byte_offset]

6

In [24]:
x[1, 2]

6



Note

The Python built-in bytes returns bytes in C-order by default which can cause confusion when trying to inspect memory layout. We use numpy.ndarray.tobytes() with order=A instead, which preserves the C or F ordering of the bytes in memory. 

In [25]:
x = np.array(
    [
        [1, 2, 3],
        [4, 5, 6]
    ],
    dtype=np.int16,
    order='C'
)

In [26]:
x.strides

(6, 2)

In [27]:
x.tobytes('A')

b'\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00'

- Need to jump 6 bytes to find the next row
- Need to jump 2 bytes to find the next column

In [28]:
y = np.array(x, order='F')
y.strides

(2, 4)

In [29]:
y.tobytes('A')

b'\x01\x00\x04\x00\x02\x00\x05\x00\x03\x00\x06\x00'


- Need to jump 2 bytes to find the next row
- Need to jump 4 bytes to find the next column


Note: Now we can understand the behavior of `.view()`: 

In [30]:
y = np.array([[1, 3], [2, 4]], dtype=np.uint8).transpose()
x = y.copy()

In [31]:
x.strides

(2, 1)

In [32]:
y.strides

(1, 2)

In [33]:
x.tobytes('A')

b'\x01\x02\x03\x04'

In [34]:
x.tobytes('A')

b'\x01\x02\x03\x04'

### Slicing with integers


- Everything can be represented by changing only shape, strides, and possibly adjusting the data pointer!
- Never makes copies of the data


In [35]:
x = np.array([1, 2, 3, 4, 5, 6], dtype=np.int32)
y = x[::-1]
y

array([6, 5, 4, 3, 2, 1])

In [36]:
y.strides

(-4,)

In [37]:
y = x[2:]
y.__array_interface__['data'][0] - x.__array_interface__['data'][0]

8

In [38]:
x = np.zeros((10, 10, 10), dtype=np.float)
x.strides

(800, 80, 8)

In [39]:
x[::2, ::3, ::4].strides

(1600, 240, 32)

Similarly, transposes never make copies (it just swaps strides):

In [40]:
x = np.zeros((10, 10, 10), dtype=np.float)
x.strides

(800, 80, 8)

In [41]:
x.T.strides

(8, 80, 800)

But: not all reshaping operations can be represented by playing with strides:

In [42]:
a = np.arange(6, dtype=np.int8).reshape(3, 2)
b = a.T
b.strides

(1, 2)

Here, there is no way to represent the array c given one stride and the block of memory for a. Therefore, the reshape operation needs to make a copy here.

In [43]:
from numpy.lib.stride_tricks import as_strided
help(as_strided)

Help on function as_strided in module numpy.lib.stride_tricks:

as_strided(x, shape=None, strides=None, subok=False, writeable=True)
    Create a view into the array with the given shape and strides.
    
    .. warning:: This function has to be used with extreme care, see notes.
    
    Parameters
    ----------
    x : ndarray
        Array to create a new.
    shape : sequence of int, optional
        The shape of the new array. Defaults to ``x.shape``.
    strides : sequence of int, optional
        The strides of the new array. Defaults to ``x.strides``.
    subok : bool, optional
        .. versionadded:: 1.10
    
        If True, subclasses are preserved.
    writeable : bool, optional
        .. versionadded:: 1.12
    
        If set to False, the returned array will always be readonly.
        Otherwise it will be writable if the original array was. It
        is advisable to set this to False if possible (see Notes).
    
    Returns
    -------
    view : ndarray
    
   

In [44]:
# as_strided does not check that you stay inside the memory block bounds… 
x = np.array([1, 2, 3, 4], dtype=np.int16)
as_strided(x, strides=(2*2, ), shape=(2, ))

array([1, 3], dtype=int16)

In [45]:
x[::2]

array([1, 3], dtype=int16)

## Broadcasting
    Doing something useful with it: outer product of [1, 2, 3, 4] and [5, 6, 7]


In [46]:
x = np.array([1, 2, 3, 4], dtype=np.int16)
x2 = as_strided(x, strides=(0, 1*2), shape=(3, 4))
x2

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4]], dtype=int16)

In [47]:
y = np.array([5, 6, 7], dtype=np.int16)
y2 = as_strided(y, strides=(1*2, 0), shape=(3, 4))
y2

array([[5, 5, 5, 5],
       [6, 6, 6, 6],
       [7, 7, 7, 7]], dtype=int16)

In [48]:
x2 * y2

array([[ 5, 10, 15, 20],
       [ 6, 12, 18, 24],
       [ 7, 14, 21, 28]], dtype=int16)

… seems somehow familiar …

In [49]:
x = np.array([1, 2, 3, 4], dtype=np.int16)
y = np.array([5, 6, 7], dtype=np.int16)
x[np.newaxis, :] * y[:, np.newaxis]

array([[ 5, 10, 15, 20],
       [ 6, 12, 18, 24],
       [ 7, 14, 21, 28]], dtype=int16)


    Internally, array broadcasting is indeed implemented using 0-strides.


### More tricks: diagonals 

__challange__: Pick diagonal entries of the matrix: (assume C memory order):

In [50]:
x = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]], dtype=np.int32)

In [51]:
x_diag = as_strided(x, shape=(3, ), strides=((3+1)*x.itemsize, ))
x_diag

array([1, 5, 9])

In [52]:
as_strided(x[0, 1:], shape=(2, ), strides=((3+1)*x.itemsize, ))

array([2, 6])

In [53]:
as_strided(x[1:, 0], shape=(2, ), strides=((3+1)*x.itemsize, ))

array([4, 8])

__challange__: Compute the tensor trace:

In [54]:
x = np.arange(5*5*5*5).reshape(5, 5, 5, 5)
s = 0
for i in range(0, 5):
    for j in range(0, 5):
        s+= x[j, i, j, i]

by striding, and using sum() on the result.

In [55]:
y = as_strided(x, shape=(5, 5), strides=((5*5*5 + 5)*x.itemsize, (5*5 + 1)*x.itemsize))
s2 = y.sum()

In [56]:
assert s == s2

## 2.2.2.2. Exercise: building an ufunc from scratch


The Mandelbrot fractal is defined by the iteration

$$ z \leftarrow z^2 + c$$

where $c = x + i y$ is a complex number. This iteration is repeated – if $z$ stays finite no matter how long the iteration runs, $c$ belongs to the Mandelbrot set.

- make a ufunc called mandel(z0, c) that computes:
    ```
    z = z0
    for k in range(0, iterations):
        z = z*z + c
    ```
    say 100 iterations or until `z.real**2 + z.imag**2 > 1000`. Use it to detemrine which c
    are in the Mandelbrot set.
- write it in Cython

In [57]:
# The elementwise function
# ------------------------

cdef void mandel_single_point(double complex *z_in, 
                              double complex *c_in,
                              double complex *z_out) nogil:
    #
    # The Mandelbrot iteration
    #

    #
    # Some points of note:
    #
    # - It's *NOT* allowed to call any Python functions here.
    #
    #   The Ufunc loop runs with the Python Global Interpreter Lock released.
    #   Hence, the ``nogil``.
    #
    # - And so all local variables must be declared with ``cdef``
    #
    # - Note also that this function receives *pointers* to the data;
    #   the "traditional" solution to passing complex variables around
    #

    cdef double complex z = z_in[0]
    cdef double complex c = c_in[0]
    cdef int k  # the integer we use in the for loop

    # Straightforward iteration

    for k in range(100):
        z = z*z + c
        if z.real**2 + z.imag**2 > 1000:
            break

    # Return the answer for this point
    z_out[0] = z


# Boilerplate Cython definitions
#
# Pulls definitions from the Numpy C headers.
# -------------------------------------------

from numpy cimport import_array, import_ufunc 
from numpy cimport (PyUFunc_FromFuncAndData,
                    PyUFuncGenericFunction)
from numpy cimport NPY_CDOUBLE
from numpy cimport PyUFunc_DD_D

# Required module initialization
# ------------------------------

import_array()
import_ufunc()


# The actual ufunc declaration
# ----------------------------

cdef PyUFuncGenericFunction loop_func[1]
cdef char input_output_types[3]
cdef void *elementwise_funcs[1]

loop_func[0] = PyUFunc_DD_D

input_output_types[0] = NPY_CDOUBLE
input_output_types[1] = NPY_CDOUBLE
input_output_types[2] = NPY_CDOUBLE

elementwise_funcs[0] = <void*>mandel_single_point

mandel = PyUFunc_FromFuncAndData(
    loop_func,
    elementwise_funcs,
    input_output_types,
    1, # number of supported input types
    2, # number of input args
    1, # number of output args
    0, # `identity` element, never mind this
    "mandel", # function name
    "mandel(z, c) -> computes iterated z*z + c", # docstring
    0 # unused
    )

"""
Plot Mandelbrot
================

Plot the Mandelbrot ensemble.

"""

import numpy as np
import mandel
x = np.linspace(-1.7, 0.6, 1000)
y = np.linspace(-1.4, 1.4, 1000)
c = x[None,:] + 1j*y[:,None]
z = mandel.mandel(c, c)

import matplotlib.pyplot as plt
plt.imshow(abs(z)**2 < 1000, extent=[-1.7, 0.6, -1.4, 1.4])
plt.gray()
plt.show()



SyntaxError: invalid syntax (<ipython-input-57-53100d327b73>, line 4)